In [1]:
import pandas as pd
data = pd.read_stata('2302_偽・誤情報調査データ.dta')
data.to_csv('2302_偽・誤情報調査データ.csv', index=False, encoding='cp932')

In [4]:
data = pd.read_csv('2302_偽・誤情報調査データ.csv', encoding='cp932')

C:\Users\tanihara\AppData\Local\Temp\ipykernel_15820\1951895915.py:1: DtypeWarning: Columns (2115,2133,2188,2229,2302,2319) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('2302_偽・誤情報調査データ.csv', encoding='cp932')


# 記述統計

In [9]:
# 事前の審議判断
pre = data['q38提示情報の信頼度'].value_counts().sort_index()
print("\n最初の真偽判断の度数分布:\n", pre)


最初の真偽判断の度数分布:
 0    15000
1      364
2      824
3      766
4     2497
5      298
6      158
7       93
Name: q38提示情報の信頼度, dtype: int64


In [11]:
# preのウェイト付き度数分布
pre_weighted = data.groupby('q38提示情報の信頼度').apply(lambda x: (x['weight']).sum())
print("\n最初の真偽判断の度数分布（ウェイト）:\n", pre_weighted)


最初の真偽判断の度数分布（ウェイト）:
 q38提示情報の信頼度
0    7323.697185
1     180.032133
2     410.860121
3     381.496355
4    1234.016197
5     146.215168
6      77.665296
7      45.640780
dtype: float64


In [13]:
# 政府
# 条件に合致するデータのフィルタリング
filtered_data = data[(data['q38提示情報の信頼度'].between(4, 7)) & (data['q39a11'] == 1) & (data['q39a14'] == 1)]

# ウェイト付きでの事後判断の度数分布の計算
post_weighted = filtered_data.groupby('q39a2').apply(lambda x: (x['weight']).sum())

# 結果の表示
print("条件を満たす被験者の事後判断のウェイト付き真偽判定の分布:\n", post_weighted)

条件を満たす被験者の事後判断のウェイト付き真偽判定の分布:
 q39a2
1      5.907315
2      7.785225
3     17.834416
4    106.111134
5     56.354166
6     57.329846
7     15.621677
dtype: float64


In [16]:
# バズフィード
# 条件に合致するデータのフィルタリング
filtered_data_b = data[(data['q38提示情報の信頼度'].between(4, 7)) & (data['q39b11'] == 1) & (data['q39b14'] == 1)]

# ウェイト付きでの事後判断の度数分布の計算
post_weighted_b = filtered_data_b.groupby('q39b2').apply(lambda x: (x['weight']).sum())

# 結果の表示
print(post_weighted_b)

q39b2
1      5.009824
2      8.787310
3     14.571898
4    101.999534
5     47.134631
6     67.229507
7     25.333127
dtype: float64


In [18]:
# マスメディア
# 条件に合致するデータのフィルタリング
filtered_data_m = data[(data['q38提示情報の信頼度'].between(4, 7)) & (data['q39c11'] == 1) & (data['q39c14'] == 1)]

# ウェイト付きでの事後判断の度数分布の計算
post_weighted_m = filtered_data_m.groupby('q39c2').apply(lambda x: (x['weight']).sum())

# 結果の表示
print(post_weighted_m)

q39c2
1     5.839909
2     8.707106
3    22.823138
4    88.208185
5    76.488528
6    70.817041
7    18.615322
dtype: float64


In [20]:
# 統制
# 条件に合致するデータのフィルタリング
filtered_data_c = data[(data['q38提示情報の信頼度'].between(4, 7))]

# ウェイト付きでの事後判断の度数分布の計算
post_weighted_c = filtered_data_c.groupby('q39d1').apply(lambda x: (x['weight']).sum())

# 結果の表示
print(post_weighted_c)

q39d1
0    1164.730122
1       8.196913
2      16.174838
3      30.411664
4     263.214440
5      17.305496
6       2.031899
7       1.472069
dtype: float64


# それぞれのファクトチェックに効果はあるか（t検定）

In [23]:
from scipy.stats import ttest_ind_from_stats
# 政府
# 介入グループのデータ
intervention_proportion = 0.484  # 48.4%
intervention_size = 266.9438
# 介入グループの成功数と失敗数
intervention_success = intervention_proportion * intervention_size
intervention_failure = (1 - intervention_proportion) * intervention_size

# 統制グループのデータ
control_proportion = 0.061  # 6.1%
control_size = 338.8073
# 統制グループの成功数と失敗数
control_success = control_proportion * control_size
control_failure = (1 - control_proportion) * control_size

# t検定の実行
# 成功数（mean）、サンプルサイズ（nobs）、標準偏差（std_dev）を使用
t_stat, p_value = ttest_ind_from_stats(
    mean1=intervention_success, std1=(intervention_success * intervention_failure / intervention_size)**0.5, nobs1=intervention_size,
    mean2=control_success, std2=(control_success * control_failure / control_size)**0.5, nobs2=control_size
)

print(f"t統計量: {t_stat:.4f}, p値: {p_value:.20f}")

t統計量: 209.1022, p値: 0.00000000000000000000


In [25]:
# バズフィード
# 介入グループのデータ
intervention_proportion = 0.517 
intervention_size = 270.0658
# 介入グループの成功数と失敗数
intervention_success = intervention_proportion * intervention_size
intervention_failure = (1 - intervention_proportion) * intervention_size

# 統制グループのデータ
control_proportion = 0.061  # 6.1%
control_size = 338.8073
# 統制グループの成功数と失敗数
control_success = control_proportion * control_size
control_failure = (1 - control_proportion) * control_size

# t検定の実行
# 成功数（mean）、サンプルサイズ（nobs）、標準偏差（std_dev）を使用
t_stat, p_value = ttest_ind_from_stats(
    mean1=intervention_success, std1=(intervention_success * intervention_failure / intervention_size)**0.5, nobs1=intervention_size,
    mean2=control_success, std2=(control_success * control_failure / control_size)**0.5, nobs2=control_size
)

print(f"t統計量: {t_stat:.4f}, p値: {p_value:.20f}")

t統計量: 228.5748, p値: 0.00000000000000000000


In [26]:
# マスメディア
# 介入グループのデータ
intervention_proportion = 0.569 
intervention_size = 291.4992
# 介入グループの成功数と失敗数
intervention_success = intervention_proportion * intervention_size
intervention_failure = (1 - intervention_proportion) * intervention_size

# 統制グループのデータ
control_proportion = 0.061  # 6.1%
control_size = 338.8073
# 統制グループの成功数と失敗数
control_success = control_proportion * control_size
control_failure = (1 - control_proportion) * control_size

# t検定の実行
# 成功数（mean）、サンプルサイズ（nobs）、標準偏差（std_dev）を使用
t_stat, p_value = ttest_ind_from_stats(
    mean1=intervention_success, std1=(intervention_success * intervention_failure / intervention_size)**0.5, nobs1=intervention_size,
    mean2=control_success, std2=(control_success * control_failure / control_size)**0.5, nobs2=control_size
)

print(f"t統計量: {t_stat:.4f}, p値: {p_value:.20f}")

t統計量: 275.6120, p値: 0.00000000000000000000


# 介入措置ごとのt検定（ボンフェローニの補正をしたt-test）

In [27]:
# 政府 vs バズフィード
# 政府グループのデータ
intervention_proportion = 0.484  
intervention_size = 266.9438
# 政府グループの成功数と失敗数
intervention_success = intervention_proportion * intervention_size
intervention_failure = (1 - intervention_proportion) * intervention_size

# バズフィードグループのデータ
control_proportion = 0.517 
control_size = 270.0658
# バズフィードグループの成功数と失敗数
control_success = control_proportion * control_size
control_failure = (1 - control_proportion) * control_size

# t検定の実行
# 成功数（mean）、サンプルサイズ（nobs）、標準偏差（std_dev）を使用
t_stat, p_value = ttest_ind_from_stats(
    mean1=intervention_success, std1=(intervention_success * intervention_failure / intervention_size)**0.5, nobs1=intervention_size,
    mean2=control_success, std2=(control_success * control_failure / control_size)**0.5, nobs2=control_size
)

print(f"t統計量: {t_stat:.4f}, p値: {p_value:.20f}")

t統計量: -14.7482, p値: 0.00000000000000000000


In [28]:
# 政府 vs マスメディア
# 政府グループのデータ
intervention_proportion = 0.484  
intervention_size = 266.9438
# 政府グループの成功数と失敗数
intervention_success = intervention_proportion * intervention_size
intervention_failure = (1 - intervention_proportion) * intervention_size

# マスメディアグループのデータ
control_proportion = 0.569   
control_size = 291.4992
# マスメディアグループの成功数と失敗数
control_success = control_proportion * control_size
control_failure = (1 - control_proportion) * control_size

# t検定の実行
# 成功数（mean）、サンプルサイズ（nobs）、標準偏差（std_dev）を使用
t_stat, p_value = ttest_ind_from_stats(
    mean1=intervention_success, std1=(intervention_success * intervention_failure / intervention_size)**0.5, nobs1=intervention_size,
    mean2=control_success, std2=(control_success * control_failure / control_size)**0.5, nobs2=control_size
)

print(f"t統計量: {t_stat:.4f}, p値: {p_value:.20f}")

t統計量: -52.0303, p値: 0.00000000000000000000


In [29]:
# バズフィード vs マスメディア
# バズフィードグループのデータ
intervention_proportion = 0.517   
intervention_size = 270.0658
# バズフィードグループの成功数と失敗数
intervention_success = intervention_proportion * intervention_size
intervention_failure = (1 - intervention_proportion) * intervention_size

# マスメディアグループのデータ
control_proportion = 0.569   
control_size = 291.4992
# マスメディアグループの成功数と失敗数
control_success = control_proportion * control_size
control_failure = (1 - control_proportion) * control_size

# t検定の実行
# 成功数（mean）、サンプルサイズ（nobs）、標準偏差（std_dev）を使用
t_stat, p_value = ttest_ind_from_stats(
    mean1=intervention_success, std1=(intervention_success * intervention_failure / intervention_size)**0.5, nobs1=intervention_size,
    mean2=control_success, std2=(control_success * control_failure / control_size)**0.5, nobs2=control_size
)

print(f"t統計量: {t_stat:.4f}, p値: {p_value:.20f}")

t統計量: -37.2549, p値: 0.00000000000000000000


# 各グループのウェイトバック前のサンプルサイズ

In [32]:
# 政府
# 条件に合致するデータのフィルタリング
filtered_data = data[(data['q38提示情報の信頼度'].between(4, 7)) & (data['q39a11'] == 1) & (data['q39a14'] == 1)]

post_g = filtered_data['q39a2'].value_counts().sort_index()

# 結果の表示
print(post_g)

1     12
2     16
3     37
4    215
5    114
6    117
7     31
Name: q39a2, dtype: int64


In [33]:
# バズフィード
# 条件に合致するデータのフィルタリング
filtered_data_b = data[(data['q38提示情報の信頼度'].between(4, 7)) & (data['q39b11'] == 1) & (data['q39b14'] == 1)]

post_b = filtered_data_b['q39b2'].value_counts().sort_index()

# 結果の表示
print(post_b)

1     10
2     18
3     29
4    206
5     97
6    137
7     51
Name: q39b2, dtype: int64


In [35]:
# マスメディア
# 条件に合致するデータのフィルタリング
filtered_data_m = data[(data['q38提示情報の信頼度'].between(4, 7)) & (data['q39c11'] == 1) & (data['q39c14'] == 1)]

post_m = filtered_data_m['q39c2'].value_counts().sort_index()

# 結果の表示
print(post_m)

1     12
2     18
3     45
4    178
5    155
6    142
7     37
Name: q39c2, dtype: int64


In [38]:
# コントロール群
# 条件に合致するデータのフィルタリング
filtered_data_c = data[(data['q38提示情報の信頼度'].between(4, 7))]

post_c = filtered_data_c['q39d1'].value_counts().sort_index()

# 結果の表示
print(post_c)

0    2360
1      16
2      33
3      61
4     536
5      33
6       4
7       3
Name: q39d1, dtype: int64
